# Hard
- Test Environment: Windows 10, py-visa, NI-VISA


## TODO:

Testing on more platforms:

- x86, ubuntu-18.04.
- raspberrypi4.
- FreeBSD.

In [2]:
import time
import mhs5200

In [3]:
signal_gen_port = "COM4"
oscilloscope_port = scope_port = 'USB0::0x1AB1::0x0588::DS1EU152500705::INSTR'

In [4]:
import mhs5200.Enums

In [5]:
with mhs5200.MHS5200(signal_gen_port) as signal_generator:
    for channel in signal_generator.channels:
        channel.amplitude=1
        channel.duty_cycle=50
        channel.frequency=1
        channel.offset=0
        channel.phase=0
        channel.wave=mhs5200.Enums.SQUARE

In [7]:
import pyvisa
rm = pyvisa.ResourceManager()
rm.list_resources()

scope = rm.open_resource(oscilloscope_port)

In [8]:
import uuid

In [9]:
class Scope():
    def __init__(self, *args, oscilloscope=None, oscilloscope_port=None):
        if oscilloscope_port is not None:
            self.resource = rm.open_resource(oscilloscope_port)
        else:
            self.resource = oscilloscope
            
        for fcn in ["write", "query", "close", "query_ascii_values"]:
            setattr(self, fcn, getattr(self.resource, fcn))
        
    def __enter__(self):
        return self
    
    def __exit__(self, type, value, tb):
        t1=time.time()
        try:
            while True:
                if time.time()>t1+5:
                    break
                self.close()
                time.sleep(0.2)
        except InvalidSession:
            pass
        except:
            raise

In [10]:
scope = Scope(oscilloscope_port=oscilloscope_port)

In [11]:
scope = Scope(oscilloscope_port=oscilloscope_port)

In [12]:
scope.close()

In [13]:
import pandas as pd

In [14]:
def test_frequency_amplitude3(frequency, amplitude, signal_gen, scope):
    for chan in signal_gen.channels:
        chan.frequency=frequency
        chan.amplitude=amplitude
        chan.phase=0
    
    period = 1/float(frequency)
    timescale="{:.20f}".format(float(period/5))
    # Configure scope
    
    scope.write(f":MEASURE:TOTAL ON")
    scope.write(f":TIMebase:SCALE {timescale}")
    for scope_channel in [1, 2]:
        scope.write(f":CHANNEL{scope_channel}:probe 1")
        scope.write(f":CHANNEL{scope_channel}:scale {amplitude/5}")
        scope.write(f":CHANNEL{scope_channel}:offset 0")
    # Configure signal generator
    for chan in signal_gen.channels:
        chan.frequency=frequency
        chan.amplitude=amplitude
        chan.offset = 0
        chan.phase=0

    df = dict()
    df["uuid"] = str(uuid.uuid4())
    df["frequency"] = frequency
    df["amplitude"] = amplitude
    for source in ["CHAN1", "CHAN2"]:
        scope.write(f":MEASURE:SOURCE {source}")
        time.sleep(1)
        for param in ['VPP',
                        'VMAX',
                        'VMIN',
                        'VAMPlitude',
                        'VTOP',
                        'VBASe',
                        'VAVerage',
                        'VRMS',
                        'OVERshoot',
                        'PREShoot',
                        'FREQuency',
                        'RISetime',
                        'FALLtime',
                        'PERiod',
                        'PWIDth',
                        'NWIDth',
                        'PDUTycycle',
                        'NDUTycycle',
                        'PDELay',
                        'NDELay',
                        'TOTal',
                        'SOURce',]:
            try:
                measured = scope.query_ascii_values(f":MEASURE:{param}?")[0]
            except:
                measured = scope.query(f":MEASURE:{param}?")[0]

            df[f"{source}_{param}"] = measured
    return pd.DataFrame(df, index=[0])

Loop through:

$$f=[1,10,100]$$
$$A=[1,10,20]$$

10x times to create statistics.

In [18]:
df = pd.DataFrame()
with mhs5200.MHS5200(signal_gen_port) as signal_gen:
    with Scope(oscilloscope_port=oscilloscope_port) as scope:
        for _ in range(10):
            for frequency in [1, 10, 100]:
                for amplitude in [1, 10, 20]:
                    result_df = test_frequency_amplitude3(frequency, amplitude, signal_gen=signal_gen, scope=scope)
                    df = df.append(result_df)

In [23]:
df.groupby(["frequency", "amplitude"]).agg(["mean", "median", "std", "var"])

CHAN1_VPP                                     \
                             mean        median       std        var   
frequency amplitude                                                    
1         1          9.900000e+37  9.900000e+37  0.000000   0.000000   
          10         9.600000e-01  9.600000e-01  0.000000   0.000000   
          20         6.744000e+00  1.060000e+01  4.978075  24.781227   
10        1          8.384000e-01  9.760000e-01  0.311368   0.096950   
          10         8.682000e+00  1.060000e+01  4.069971  16.564662   
          20         2.100000e+01  2.100000e+01  0.000000   0.000000   
100       1          9.816000e-01  9.840000e-01  0.005400   0.000029   
          10         1.073000e+01  1.065000e+01  0.170294   0.029000   
          20         2.155000e+01  2.120000e+01  0.581664   0.338333   

                       CHAN1_VMAX                                    \
                             mean        median       std       var   
frequency amplitude                                                   
1         1          9.900000e+37  9.900000e+37  0.000000  0.000000   
          10         4.800000e-01  4.800000e-01  0.000000  0.000000   
          20         3.360000e+00  5.280000e+00  2.478709  6.144000   
10        1          4.488000e-01  4.960000e-01  0.149260  0.022278   
          10         4.320000e+00  5.280000e+00  2.023858  4.096000   
          20         1.060000e+01  1.060000e+01  0.000000  0.000000   
100       1          4.984000e-01  4.960000e-01  0.003864  0.000015   
          10         5.384000e+00  5.320000e+00  0.136235  0.018560   
          20         1.095000e+01  1.070000e+01  0.350397  0.122778   

                       CHAN1_VMIN                    ...       CHAN2_NWIDth  \
                             mean        median      ...                std   
frequency amplitude                                  ...                      
1         1          9.900000e+37  9.900000e+37      ...       4.174207e+37   
          10        -4.800000e-01 -4.800000e-01      ...       5.112338e+37   
          20        -3.360000e+00 -5.280000e+00      ...       4.174207e+37   
10        1         -4.344000e-01 -4.800000e-01      ...       1.264911e-04   
          10        -4.336000e+00 -5.280000e+00      ...       0.000000e+00   
          20        -1.040000e+01 -1.040000e+01      ...       0.000000e+00   
100       1         -4.832000e-01 -4.800000e-01      ...       0.000000e+00   
          10        -5.368000e+00 -5.360000e+00      ...       1.264911e-05   
          20        -1.061000e+01 -1.050000e+01      ...       1.264911e-05   

                                  CHAN2_PDUTycycle                       \
                              var             mean median           std   
frequency amplitude                                                       
1         1          1.742400e+75     1.980000e+37  0.502  4.174207e+37   
          10         2.613600e+75     3.960000e+37  0.504  5.112338e+37   
          20         1.742400e+75     1.980000e+37  0.501  4.174207e+37   
10        1          1.600000e-08     4.996000e-01  0.500  1.264911e-03   
          10         0.000000e+00     5.000000e-01  0.500  0.000000e+00   
          20         0.000000e+00     5.000000e-01  0.500  0.000000e+00   
100       1          0.000000e+00     5.000000e-01  0.500  0.000000e+00   
          10         1.600000e-10     4.996000e-01  0.500  1.264911e-03   
          20         1.600000e-10     4.996000e-01  0.500  1.264911e-03   

                                  CHAN2_NDUTycycle                       \
                              var             mean median           std   
frequency amplitude                                                       
1         1          1.742400e+75     1.980000e+37  0.500  4.174207e+37   
          10         2.613600e+75     3.960000e+37  0.501  5.112338e+37   
          20         1.742400e+75     1.980000e+37  0.500  4.174207e+37   
10    

In [45]:
import numpy as np

In [51]:
df = pd.DataFrame()
with mhs5200.MHS5200(signal_gen_port) as signal_gen:
    with Scope(oscilloscope_port=oscilloscope_port) as scope:
        for _ in range(2):
            for frequency in np.logspace(np.log10(1000), np.log10(1000000), 2):
                for amplitude in [20]:
                    result_df = test_frequency_amplitude3(frequency, amplitude, signal_gen=signal_gen, scope=scope)
                    df = df.append(result_df)

In [52]:
df.groupby(["frequency"])["CHAN1_FREQuency"].agg(["mean", "median", "std", "var"])

,mean,median,std,var
frequency,,,,
1000.0,1000.0,1000.0,0.0,0.0
1000000.0,1000000.0,1000000.0,0.0,0.0


In [53]:
df.groupby(["frequency", "amplitude"])["CHAN2_FREQuency"].agg(["mean", "median", "std", "var"])

,,mean,median,std,var
frequency,amplitude,,,,
1000.0,20,1000.0,1000.0,0.0,0.0
1000000.0,20,1000000.0,1000000.0,0.0,0.0


In [54]:
df = pd.DataFrame()
with mhs5200.MHS5200(signal_gen_port) as signal_gen:
    with Scope(oscilloscope_port=oscilloscope_port) as scope:
        for _ in range(2):
            for frequency in np.logspace(np.log10(1000000), np.log10(100000000), 10):
                for amplitude in [20]:
                    result_df = test_frequency_amplitude3(frequency, amplitude, signal_gen=signal_gen, scope=scope)
                    df = df.append(result_df)
df.groupby(["frequency"])["CHAN1_FREQuency"].agg(["mean", "median", "std", "var"])

,mean,median,std,var
frequency,,,,
1.000000e+06,1000000.0,1000000.0,0.000000,0.0
1.778279e+06,1775000.0,1775000.0,7071.067812,50000000.0
3.162278e+06,3160000.0,3160000.0,28284.271247,800000000.0
5.623413e+06,5650000.0,5650000.0,0.000000,0.0
1.000000e+07,9785000.0,9785000.0,21213.203436,450000000.0
